In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import math 
import collections
import openpyxl
import matplotlib.pyplot as plt
import io
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import pypsa



network = pypsa.Network()
hours_in_2015 = pd.date_range('2015-01-01T00:00Z','2015-12-31T23:00Z', freq='H')
network.set_snapshots(hours_in_2015)
network.add("Bus","electricity bus")
#print(network.snapshots) 


# load electricity demand data
df_elec = pd.read_csv('time_series_60min_singleindex.csv', sep=',', index_col=0) # in MWh
df_elec.index = pd.to_datetime(df_elec.index) #change index to datatime
#print(df_elec['GB_GBN_wind_generation_actual'].head())

# add load to the bus
network.add("Load",
            "load", 
            bus="electricity bus", 
            p_set=df_elec['GB_GBN_load_actual_entsoe_transparency'])


print(network.loads_t.p_set)

def annuity(n,r):
    """Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20,0.05)*20 = 1.6"""

    if r > 0:
        return r/(1. - 1./(1.+r)**n)
    else:
        return 1/n

# add the different carriers, only gas emits CO2
network.add("Carrier", "onshorewind") # in t_CO2/MWh_th
network.add("Carrier", "offshorewind")
network.add("Carrier", "solar")
network.add("Carrier", "gas", co2_emissions=0.185)

# add onshore wind generator
df_onshorewind = pd.read_csv('onshore_wind_1979-2017.csv', sep=';', index_col=0)
df_onshorewind.index = pd.to_datetime(df_onshorewind.index)
CF_wind = df_onshorewind['GBR'][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]] #https://www.grantthornton.co.uk/globalassets/1.-member-firms/united-kingdom/pdf/documents/renewable-energy-discount-rate-survey-results-2018.pdf
#CF_wind = np.load("CF_wind_2011.npy") #used for part c
#np.save("CF_wind_2011.npy",CF_wind) #used for part c
capital_cost_onshorewind = annuity(25,0.08)*52910*(1) # in €/MW  https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/911817/electricity-generation-cost-report-2020.pdf
network.add("Generator",
            "onshorewind",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="onshorewind",
            #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_onshorewind,
            marginal_cost = 0,
            p_max_pu = CF_wind)

# add offshore wind generator
df_offshorewind = pd.read_csv('offshore_wind_1979-2017.csv', sep=';', index_col=0)
df_offshorewind.index = pd.to_datetime(df_offshorewind.index)
CF_wind_offshore = df_offshorewind['GBR'][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
#CF_wind_offshore = np.load("CF_wind_offshore_2011.npy") #used for part c
#np.save("CF_wind_offshore_2011.npy",CF_wind_offshore) #used for part c
capital_cost_offshorewind = annuity(25,0.0875)*65560*(1) # in €/MW 910000
network.add("Generator",
            "offshorewind",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="offshorewind",
            #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_offshorewind,
            marginal_cost = 0,
            p_max_pu = CF_wind_offshore)

# add solar PV generator
df_solar = pd.read_csv('pv_optimal.csv', sep=';', index_col=0)
df_solar.index = pd.to_datetime(df_solar.index)
CF_solar = df_solar['GBR'][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
#CF_solar = np.load("CF_solar_2011.npy")
#np.save("CF_solar_2011.npy",CF_solar)

capital_cost_solar = annuity(25,0.07)*50610*(1) # in €/MW 
network.add("Generator",
            "solar",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="solar",
            #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_solar,
            marginal_cost = 0,
            p_max_pu = CF_solar)



# add OCGT (Open Cycle Gas Turbine) generator
capital_cost_OCGT = annuity(25,0.07)*930700*(1) # in €/MW #https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/315717/coal_and_gas_assumptions.PDF
fuel_cost = 14.71 # in €/MWh_th #https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1107499/quarterly_energy_prices_uk_september_2022.pdf
efficiency = 0.38 #https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/387566/Technical_Assessment_of_the_Operation_of_Coal_and_Gas_Plant_PB_Power_FIN....pdf
marginal_cost_OCGT = fuel_cost/efficiency # in €/MWh_el
network.add("Generator", 
            "OCGT",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="gas",
            #p_nom_max=1000,
            capital_cost = capital_cost_OCGT,
            marginal_cost = marginal_cost_OCGT)

network.lopf(network.snapshots, 
             pyomo=False,
             solver_name='gurobi')

#print((network.objective/network.loads_t.p.sum())) # €/MWh
print (network.generators_t.p_max_pu) # in MW

In [ ]:
#Adding CO2 constraint. Multiple points are taken into excel sheet to plot figure 4 and 5 in the report
co2_limit=0 #tonCO2
network.add("GlobalConstraint",
            "co2_limit",
            type="primary_energy",
            carrier_attribute="co2_emissions",
            sense="<=",
            constant=co2_limit)
print(network.global_constraints.constant) #CO2 limit (constant in the constraint)
network.lopf(network.snapshots, 
             pyomo=False,
             solver_name='gurobi')

print ((network.generators.p_nom_opt)/1000) # in GW